Let's start creating a dummy dataset with Function data

In [1]:
import numpy as np
import random as rnd

funData = []
resultData = []

for x in range(1,100001):
    funData.append([x,x+1,x+2])
    resultData.append([(x+x+1+x+2)*2])

print("Dataset size:")
print(np.array(funData).shape)
print(np.array(resultData).shape)

Dataset size:
(100000, 3)
(100000, 1)


In [2]:
#1.scaling data
from sklearn.preprocessing import StandardScaler
scaler_features = StandardScaler()
X_scaled = scaler_features.fit_transform(funData)
scaler_labels = StandardScaler()
Y_scaled = scaler_labels.fit_transform(resultData)
#X_scaled = np.array(funData)
#Y_scaled = np.array(resultData)

In [3]:
#2. splitting into train and test
train_size_perc = 0.9
train_limit = len(X_scaled) * train_size_perc
X_scaled_train = X_scaled[:int(train_limit)]
X_scaled_test = X_scaled[int(train_limit):]
Y_scaled_train = Y_scaled[:int(train_limit)]
Y_scaled_test = Y_scaled[int(train_limit):]

In [4]:
window = 5
features_reshaped_train = []
features_reshaped_test = []
labels_reshaped_train = []
labels_reshaped_test = []

#resizing X_scaled_train sliding window from the beginning to the end
for i in range(0, len(X_scaled_train) - window +1):
    #appending sub list of window size
    features_reshaped_train.append(X_scaled_train[i:i + window][:,[0,1,2]])
    #feeding labels with third features of the last window item. Optional
    #labels.append(Y_scaled[i + window - 1])

#resizing X_scaled_test sliding window from the beginning to the end
for i in range(0, len(X_scaled_test) - window +1):
    #appending sub list of window size
    features_reshaped_test.append(X_scaled_test[i:i + window][:,[0,1,2]])
    #feeding labels with third features of the last window item. Optional
    #labels.append(Y_scaled[i + window - 1])

#resizing Y_scaled_train sliding window from the beginning to the end
for i in range(0, len(Y_scaled_train) - window + 1):
    #appending sub list of window size
    labels_reshaped_train.append(Y_scaled_train[i + window - 1])
    #feeding labels with third features of the last window item. Optional
    #labels.append(Y_scaled[i + window - 1])

#resizing Y_scaled_test sliding window from the beginning to the end
for i in range(0, len(Y_scaled_test) - window +1):
    #appending sub list of window size
    labels_reshaped_test.append(Y_scaled_test[i + window - 1])
    #feeding labels with third features of the last window item. Optional
    #labels.append(Y_scaled[i + window - 1])
    
print(np.array(features_reshaped_train).shape)
print(np.array(features_reshaped_test).shape)
print(np.array(labels_reshaped_train).shape)
print(np.array(labels_reshaped_test).shape)

(89996, 5, 3)
(9996, 5, 3)
(89996, 1)
(9996, 1)


Now let's create an ANN predicting Data

In [11]:
from tensorflow import keras
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

def create_model(window):
    model = Sequential()

    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(window,3)))
    model.add(Dropout(0.2))
    model.add(LSTM(50, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(50))
    model.add(Dense (1))
    model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])
    return model

In [12]:
model = create_model(window)

X_train = np.array(features_reshaped_train)
Y_train = np.array(labels_reshaped_train)
X_test = np.array(features_reshaped_test)
Y_test = np.array(labels_reshaped_test)

model.fit(X_train, Y_train, batch_size=128,epochs=3,verbose=1, validation_data = (X_test, Y_test))

Train on 89996 samples, validate on 9996 samples
Epoch 1/3
89996/89996 [==============================] - 31s 345us/step - loss: 0.0189 - acc: 0.0000e+00 - val_loss: 0.0013 - val_acc: 0.0000e+00
Epoch 2/3
89996/89996 [==============================] - 28s 310us/step - loss: 0.0015 - acc: 0.0000e+00 - val_loss: 0.0109 - val_acc: 0.0000e+00
Epoch 3/3
89996/89996 [==============================] - 29s 318us/step - loss: 0.0011 - acc: 0.0000e+00 - val_loss: 0.0084 - val_acc: 0.0000e+00


In [14]:
test_data = [[6000,6001,6002],[6001,6002,6003],[6002,6003,6004],[6003,6004,6005],[6004,6005,6006]]

test_data = scaler_features.transform(test_data)

test_predict = scaler_labels.inverse_transform(model.predict(test_data.reshape(1,5,3)))
print(test_predict)

[[ 38378.546875]]
